# Incomplete regression neural network (RNN) that adopts a more complex architecture to improve on the simpler regression network previously made. 

Since the data to be deconvoluted are not the standalone distribution curves, but a series of them with one evolving into the next, an RNN can be used to analyse this whole series as a three-dimensional dataset. This has yet to be tested since making representative simulated data is not an easy task.

For descriptions of the classes and functions see the regression neural network.

In [3]:
import numpy as np

In [20]:
class Normal_Distribution:
    def __init__(self,mean,std,height,x):
        self.initialize_variables(mean,std,height,x)
        self.create_distribution()
    def initialize_variables(self,mean,std,height,x):
        self.mean = mean
        self.std = std
        self.x = x
        self.height = height
        self.data = np.array([])
    def create_distribution(self):
        self.data = self.height * np.exp(-((self.x - self.mean) ** 2) / (2 * (self.std ** 2)))

In [21]:
class Combine_Distribution:
    def __init__(self, distributions):
        self.initialize_variables(distributions)
        self.combine()
        
    def initialize_variables(self, distributions):
        self.distributions = distributions
        self.curve = np.zeros(len(self.distributions[0].data))
        
    def combine(self):
        for i in self.distributions:
            self.curve = [sum(x) for x in list(zip(self.curve,i.data))]

In [22]:
class Create_Dataset:
    def __init__(self,x,sample_number):
        self.initialize_variables(x,sample_number)
        self.create_samples()
    def initialize_variables(self,x,sample_number): 
        self.x = x
        self.sample_number = sample_number
        self.number_of_gaussians = 0
        self.name_dictionary = {}
        self.mean = 0
        self.std = 0
        self.height = 0 
        self.features = []
        self.targets = []
        self.means_array = np.zeros(10)
        self.std_array = np.zeros(10)
        self.height_array = np.zeros(10)
    def create_samples(self):
        for _ in range(0,self.sample_number):
            #self.number_of_gaussians = np.random.randint(1,10)
            self.number_of_gaussians = 1
            for i in range(0,self.number_of_gaussians):
                if self.means_array[i - 1] != 0:
                    prev_mean = self.means_array[i - 1]
                    self.mean = prev_mean + np.round(prev_mean + 3*np.random.random_sample(), 3)
                    self.means_array[i] = self.mean
                    
                else:
                    self.mean = np.round(40*np.random.random_sample(), 3)
                    self.means_array[i] = self.mean
                self.std = np.round(4*np.random.random_sample(), 3)
                self.std_array[i] = self.std
                self.height = np.round(np.random.random_sample(), 3)
                self.height_array[i] = self.height
                self.name_dictionary['distibution' + str(i)] = Normal_Distribution(self.mean,self.std,self.height,self.x)
            self.features.append(Combine_Distribution([self.name_dictionary[i].data for i in self.name_dictionary]).curve)
            self.targets.append(np.array([self.means_array,self.std_array,self.height_array]).flatten())
            self.means_array = np.zeros(10)
            self.std_array = np.zeros(10)
            self.height_array = np.zeros(10)  
            self.name_dictionary = {}
            

In [66]:
class Test_CIU:
    def __init__(self, population_number, slice_number, x, dataset_size):
        self.initialize_variables(population_number, slice_number, x, dataset_size)
        self.create_dataset()
    def initialize_variables(self, population_number, slice_number, x):
        self.dataset_size = dataset_size
        self.population_number = population_number
        self.slice_number = slice_number
        self.number_of_gaus = 0
        self.x = x
        self.quartile = len(self.x)/4
        self.mean_matrix = np.zeros((self.population_number, self.slice_number))
        self.std_matrix = np.round(np.random.rand(self.population_number, self.slice_number) * 3, 3)
        self.height_matrix = np.zeros((self.sample_number, self.slice_number))
        self.data = []
        self.targets = np.zeros((self.dataset_size ,self.slice_number ,3*self.population_number))
    def first_dist(self):
        self.number_of_gaus = np.random.randint(1,3)
        for i in range(0, self.number_of_gaus):
            self.std_matrix[0][i] = np.round(4*np.random.random_sample(), 3)
            self.height_matrix[0][i] = np.round(np.random.random_sample(), 3)
            if self.mean_matrix[0][0] == 0.:
                self.mean_matrix[0][0] = np.round(self.quartile + (len(self.x) / 2) * np.random.random_sample() , 3)
            else:
                prev_mean = self.mean_matrix[0][-1]
                if prev_mean + 6 >= len(self.x):
                    self.number_of_gaus = i
                    break
                self.mean_matrix[0][i] = np.round(3 + prev_mean + 3 * np.random.random_sample(), 3)         
    def create_CIU(self):
        self.first_dist()
        for i in range(1, self.slice_number):
            self.number_of_gaus = len([i for i in self.mean_matrix[i-1] if i != 0.])
            if np.random.randint() > 0.5:
                self.number_of_gaus += 1
                self.mean_matrix[i][self.number_of_gaus - 1] = np.round(3 + self.mean_matrix[i][self.number_of_gaus - 2] + 3 * np.random.random_sample(), 3)
            for j in range(0, self.number_of_gaus):
                if any(height_matrix[:][j]) == 1.:
                    height_matrix[i][j] =  height_matrix[i-1][j] - 0.4*np.round(np.random.random_sample(), 3)
                    if height_matrix[i][j] < 0.:
                        height_matrix[i][j] = 0.
                else:
                    height_matrix[i][j] =  height_matrix[i-1][j] + 0.4*np.round(np.random.random_sample(), 3)
                    if height_matrix[i][j] > 1.:
                        height_matrix[i][j] = 1.
    def create_dataset(self):
        for k in range(0, self.dataset_size):
            self.create_CIU()
            parameter_matrix = np.array([self.mean_matrix, self.std_matrix, self.height_matrix])
            gaussian_matrix = np.zeros((self.population_number, self.slice_number))
            CIU_matrix = np.zeros((self.slice_number, self.dataset_size))
            for i in range(0, len(parameter_matrix[:][0])):
                for j in range(0, len(parameter_matrix[:][:][0])):
                    gaussian_matrix[i][j] = Normal_Distribution(parameter_matrix[0][i][j],parameter_matrix[1][i][j],parameter_matrix[2][i][j], self.x)
                    self.targets[k][i] = np.concatenate(self.mean_matrix[j], self.std_matrix[j], self.height_matrix[j])
            for i in range(gaussian_matrix):
                CIU_matrix[k][i] = Combine_Distribution(i)
            self.data.append(CIU_matrix)
                
                    
            
        
                
                
            

In [65]:
a = np.zeros((3,5))
b = np.zeros((3,5))
c = np.array([a,b])
print(c.shape)
d = [c[:][i][:].flatten() for i in range(len(c[:][:][0]))]
print(d.shape)

(2, 3, 5)


IndexError: index 2 is out of bounds for axis 0 with size 2

In [23]:
x_range = np.linspace(0, 40, 200)

In [56]:
Dataset = Create_Dataset(x_range, 100000)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':


In [57]:
from sklearn.model_selection import train_test_split
train_x , test_x , train_y , test_y = train_test_split(Dataset.features,Dataset.targets,test_size = 0.25)

In [58]:
np.array(train_x).shape

(75000, 200)

In [59]:
from keras import *
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(300, input_shape=(200,), activation='relu'))
model.add(Dense(400, input_shape=(300,), activation='relu'))
model.add(Dense(30, input_shape=(400,)))

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(np.array(train_x), np.array(train_y), epochs=60, validation_split=0.1, batch_size=100)

Train on 67500 samples, validate on 7500 samples
Epoch 1/60
67500/67500 [==============================] - 3s 45us/step - loss: 1.1604 - val_loss: 0.1652
Epoch 2/60
67500/67500 [==============================] - 3s 42us/step - loss: 0.1300 - val_loss: 0.1045
Epoch 3/60
67500/67500 [==============================] - 3s 43us/step - loss: 0.0929 - val_loss: 0.0849
Epoch 4/60
67500/67500 [==============================] - 3s 43us/step - loss: 0.0774 - val_loss: 0.0727
Epoch 5/60
67500/67500 [==============================] - 3s 44us/step - loss: 0.0667 - val_loss: 0.0608
Epoch 6/60
67500/67500 [==============================] - 3s 44us/step - loss: 0.0595 - val_loss: 0.0548
Epoch 7/60
67500/67500 [==============================] - 3s 49us/step - loss: 0.0539 - val_loss: 0.0507
Epoch 8/60
67500/67500 [==============================] - 3s 47us/step - loss: 0.0490 - val_loss: 0.0764
Epoch 9/60
67500/67500 [==============================] - 3s 44us/step - loss: 0.0472 - val_loss: 0.0505
Epoch 

In [1]:
model.predict(np.array(test_x[0]).reshape(1,200))


NameError: name 'model' is not defined

In [ ]:
test_y[5]

In [ ]:
from matplotlib import pyplot as plt

In [ ]:

for i in range(5, len(test_y), 500):
    gaussdic = {}
    testclean = []
    predclean = []
    for j in range(0,len(test_y[i])):
        if test_y[i][j] != 0.:
            testclean.append(test_y[i][j])
            predclean.append(model.predict(np.array(test_x[i]).reshape(1,200))[0][j])
    num = int(len(testclean)/3)
    print(num)
    testcurve = Combine_Distribution([Normal_Distribution(testclean[j],testclean[j + num],testclean[j + 2*num], x_range)  for j in range(0, num)])
    predcurve = Combine_Distribution([Normal_Distribution(predclean[j],predclean[j + num],predclean[j + 2*num], x_range)  for j in range(0, num)])
    for j in range(0, len(testcurve.distributions)):
        plt.plot(np.linspace(0, 40, 200), testcurve.distributions[j].data, 'r-', label=str(i))
        plt.plot(np.linspace(0, 40, 200), predcurve.distributions[j].data, 'k-', label=str(i))
        plt.legend()
    #plt.plot(np.linspace(0, 40, 200), testcurve.curve)
    #plt.plot(np.linspace(0, 40, 200), predcurve.curve)
    plt.show()
        